In [ ]:
!pip install beautifulsoup4
!pip install requests-html

In [2]:
import sys
sys.path.append("src")
%load_ext autoreload
import pandas as pd
from scraping import scan_swedish_legislation_parallel
from vector_db import VectorDatabaseWeaviate
from embedd import split_text, EmbedderOpenAI


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = scan_swedish_legislation_parallel()
df.to_pickle("data/laws.pickle")

100%|███████████████████████████████████████| 4539/4539 [01:42<00:00, 44.37it/s]


In [3]:
df.to_pickle("data/laws.pickle")

In [2]:
df = pd.read_pickle("data/laws.pickle")

In [ ]:
isinstance()

In [3]:
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  url="http://weaviate:8080"
  # embedded_options=EmbeddedOptions()
)

In [100]:
d = openai.Embedding.create(input=["hejsan", "hejsan på dig"], model="text-embedding-ada-002")

In [ ]:
for d in embedder:
    print(d)

In [236]:
class_obj = VectorDatabaseWeaviate.setup_class_object_structure(
    class_name="Swedish_legislation",
    names=[
        "SFS_number", "title", "issuer", "issued_date", "in_effect_date", "content"
    ],
    data_types=["text"] * 6,
    description="Swedish legislation"
)

AttributeError: type object 'VectorDatabaseWeaviate' has no attribute 'setup_class_object_structure'

In [172]:
client.schema.create_class(class_obj)

In [151]:
df = pd.read_pickle("data/laws.pickle").sample(2)

embedder = EmbedderOpenAI(
    df=df,
    embedd_column="content",
    text_max_length=2000,
    text_overlap=100,
    model_name="text-embedding-ada-002",
    context_columns=["title"],
)

embedder.setup()

In [135]:
embedder._df.n_tokens.max()

42

In [136]:
embedder.embedd(43)

In [ ]:
df = pd.DataFrame(outputs)
df['content'] = df['content'].apply(lambda x: split_text(str(x), 15000, 200))
df = df.explode('content')

In [8]:
df

,SFS_number,title,issuer,issued_date,in_effect_date,content
0,2023:196,Lag (2023:196) om kommuners ansvar för brottsf...,Departement:Justitiedepartementet L5,2023-04-20,Ikraft:2023-07-01 överg.best.,/Träder i kraft I:2023-07-01/\r\nLagens innehå...
1,2023:179,Förordning (2023:179) om statsbidrag för perso...,Departement:Utbildningsdepartementet,2023-04-05,Ikraft:2023-05-10 överg.best.,/Träder i kraft I:2023-05-10/\r\nInledande bes...
2,2023:168,Förordning (2023:168) om statsbidrag för kvinn...,Departement:Arbetsmarknadsdepartementet,2023-03-30,Ikraft:2023-05-03 överg.best.,/Träder i kraft I:2023-05-03/\r\nInledande bes...
3,2023:144,Förordning (2023:144) om statsbidrag för fortb...,Departement:Utbildningsdepartementet,2023-03-16,Ikraft:2023-07-01 överg.best.,/Träder i kraft I:2023-07-01/\r\nInledande bes...
4,2023:133,Förordning (2023:133) om producentansvar för däck,Departement:Klimat- och näringslivsdepartementet,2023-03-16,Ikraft:2024-01-01 överg.best.,/Träder i kraft I:2024-01-01/\r\n1 § Denna för...
...,...,...,...,...,...,...
4534,1851:55,/r1/ Kungl. Maj:ts Förordning (1851:55 s.4)\r\...,Departement:Justitiedepartementet L2,1851-12-20,"Hafwa flera förbundit sig att, en för alla och...",NaN
4535,1845:50,"Lag (1845:50 s.1) om handel med lösören, som k...",Departement:Justitiedepartementet L2,1845-11-20,Ändring införd:t.o.m. SFS 2022:970,"1 § Vill den, som tillhandlar sig lösören och ..."
4536,1810:0926,Successionsordning (1810:0926);,Departement:Justitiedepartementet L6,1810-09-26,Omtryck:SFS 1979:935,"Vi CARL, med Guds nåde, Sveriges, Götes och Ve..."
4537,1810:0403,Förordning (1810:0403) huruledes rättelse och ...,Departement:Justitiedepartementet L2,1810-04-03,Ändring införd:t.o.m. SFS 1972:206,"Vi Carl med Guds nåde, Sveriges, Götes och Wen..."


In [232]:
from typing import Dict, Any

class VectorDatabase(object):
    
    def __init__(self, name: str):
        
        self._name = name
        
    def _exists(self):
        pass
    
class VectorDatabaseWeaviate(VectorDatabase):

    def __init__(
        self,
        name,
        url: str,
        schema: Dict[str, Any]
    ):
        
        super().__init__(
            name=name,
        )
        
        assert name[0].isupper(), "First letter is not capitalized, Weaviate will complain..."
        
        self._client = weaviate.Client(
          url=url
        )
        
        if self.exists is False:
            self._setup_class(schema)
            
    def _setup_class(self, schema):
        
        assert schema["class"] == self._name
        
        self._client.schema.create_class(schema)
        
    @staticmethod
    def _setup_class_object_structure(
        class_name: str,
        names: List[str],
        data_types: List[str],
        description: str,
    ):

        properties = [
            {
                "name": name,
                "dataType": [dataType],
            }
            for name, dataType in zip(names, data_types)
        ]

        class_obj = {
            "class": class_name,
            "vectorizer": "none", # we are providing the vectors so this field is none
            "description": description,
            "properties": properties
        }

        return class_obj
        
    @property
    def exists(self):
    
        current_schema = client.schema.get()
        
        exists = False
        
        for class_obj in current_schema['classes']:
           
            class_name = class_obj['class']
            
            if self._name == class_name:
                exists = True
                break
            
        return exists

In [227]:
class_obj

{'class': 'Swedish_legislation',
 'vectorizer': 'none',
 'description': 'Swedish legislation',
 'properties': [{'name': 'SFS_number', 'dataType': ['text']},
  {'name': 'title', 'dataType': ['text']},
  {'name': 'issuer', 'dataType': ['text']},
  {'name': 'issued_date', 'dataType': ['text']},
  {'name': 'in_effect_date', 'dataType': ['text']},
  {'name': 'content', 'dataType': ['text']}]}

In [233]:
vector_db = VectorDatabaseWeaviate(
    name="Swedish_legislation",
    url="http://weaviate:8080",
    schema=class_obj,
)

Test
Post
Swedish_leg
Swedish_legislation
True
Test
Post
Swedish_leg
Swedish_legislation


In [195]:
vector_db._exists()

Test
Post
Swedish_leg


[{'class': 'Test',
  'description': 'holds all laws',
  'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
   'cleanupIntervalSeconds': 60,
   'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
  'properties': [{'dataType': ['text'],
    'name': 'content',
    'tokenization': 'word'},
   {'dataType': ['text'], 'name': 'description', 'tokenization': 'word'},
   {'dataType': ['number[]'],
    'description': "This property was generated by Weaviate's auto-schema feature on Thu Apr 27 18:40:55 2023",
    'name': '_vector'},
   {'dataType': ['number[]'],
    'description': "This property was generated by Weaviate's auto-schema feature on Thu Apr 27 18:45:05 2023",
    'name': 'vector'}],
  'replicationConfig': {'factor': 1},
  'shardingConfig': {'virtualPerPhysical': 128,
   'desiredCount': 1,
   'actualCount': 1,
   'desiredVirtualCount': 128,
   'actualVirtualCount': 128,
   'key': '_id',
   'strategy': 'hash',
   'function': 'murmur3'},
  'vectorIndexConfig': {'ski

In [168]:
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  url="http://weaviate:8080"
  # embedded_options=EmbeddedOptions()
)

In [7]:
current_schema = client.schema.get()
for class_obj in current_schema['classes']:
    class_name = class_obj['class']
    client.schema.delete_class(class_name)

In [9]:
current_schema = client.schema.get()
current_schema

{'classes': []}

In [11]:
class_obj = {
    "class": "Test",
    "vectorizer": "none", # we are providing the vectors so this field is none
    "description": "holds all laws",
    "properties": [
        {
            "name": "content",
            "dataType": ["text"],
        },
        {
            "name": "description",
            "dataType": ["text"],
        },
    ]
}

client.schema.create_class(class_obj)

In [36]:
article_1 = {
    "content": "Example Article 1",
    "description": "This is an example article about machine learning and artificial intelligence.",
}

article_2 = {
    "content": "Example Article 2",
    "description": "This is another example article about natural language processing and its applications.",
}

In [45]:
with client.batch as batch:  # Context manager manages batch flushing
    batch.batch_size=1
    batch.dynamic=True
    for data_obj in [article_1, article_2]:
        batch.add_data_object(data_obj, "Test", vector=[-0.1, -0.2, -0.3, -0.4, -0.5])

In [46]:
client.query.aggregate("Test").with_meta_count().do()

{'data': {'Aggregate': {'Test': [{'meta': {'count': 12}}]}}}

In [47]:
search_vector = [0.6] * 5

results = client.query.get(
    "Test", ["content", "description"]
).with_near_vector(
    {"vector": search_vector}
).with_additional(
    ["distance", "id", "vector"]
).with_limit(10).do()  # We use a limit of 10 in the video - lowered here for brevity

In [48]:
results

{'data': {'Get': {'Test': [{'_additional': {'distance': 1.9045341,
      'id': '141c5f74-f4f8-4756-947c-816af9e1635e',
      'vector': [-0.1, -0.2, -0.3, -0.4, -0.5]},
     'content': 'Example Article 2',
     'description': 'This is another example article about natural language processing and its applications.'},
    {'_additional': {'distance': 1.9045341,
      'id': 'e9499360-4735-497b-9efd-e95ae7420f21',
      'vector': [-0.1, -0.2, -0.3, -0.4, -0.5]},
     'content': 'Example Article 1',
     'description': 'This is an example article about machine learning and artificial intelligence.'},
    {'_additional': {'distance': 1.9045341,
      'id': 'a4470b47-35eb-4efd-bfea-782b4a6b94fb',
      'vector': [-0.1, -0.2, -0.3, -0.4, -0.5]},
     'content': 'Example Article 2',
     'description': 'This is another example article about natural language processing and its applications.'},
    {'_additional': {'distance': 1.9045341,
      'id': 'afa199db-d243-45db-8cb7-2ca79a38e355',
      'v

In [14]:
client.batch.create_objects([article_1, article_2], ["Test", "Test"])

TypeError: Batch.create_objects() takes 1 positional argument but 3 were given

In [ ]:
data_obj = {
    "content": doc
}
client.data_object.create(
    data_obj,
    "Post",
    generate_uuid('Post',doc),
    vector= vec,
)

In [7]:
schema = client.schema.get()
print(schema)

{'classes': [{'class': 'Post', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'properties': [{'dataType': ['text'], 'name': 'content', 'tokenization': 'word'}], 'replicationConfig': {'factor': 1}, 'shardingConfig': {'virtualPerPhysical': 128, 'desiredCount': 1, 'actualCount': 1, 'desiredVirtualCount': 128, 'actualVirtualCount': 128, 'key': '_id', 'strategy': 'hash', 'function': 'murmur3'}, 'vectorIndexConfig': {'skip': False, 'cleanupIntervalSeconds': 300, 'maxConnections': 64, 'efConstruction': 128, 'ef': -1, 'dynamicEfMin': 100, 'dynamicEfMax': 500, 'dynamicEfFactor': 8, 'vectorCacheMaxObjects': 1000000000000, 'flatSearchCutoff': 40000, 'distance': 'cosine', 'pq': {'enabled': False, 'bitCompression': False, 'segments': 0, 'centroids': 256, 'encoder': {'type': 'kmeans', 'distribution': 'log-normal'}}}, 'vectorIndexType': 'hnsw', 'vectorizer': 'none'}]}


In [11]:
schema["classes"]

[{'class': 'Post',
  'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
   'cleanupIntervalSeconds': 60,
   'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
  'properties': [{'dataType': ['text'],
    'name': 'content',
    'tokenization': 'word'}],
  'replicationConfig': {'factor': 1},
  'shardingConfig': {'virtualPerPhysical': 128,
   'desiredCount': 1,
   'actualCount': 1,
   'desiredVirtualCount': 128,
   'actualVirtualCount': 128,
   'key': '_id',
   'strategy': 'hash',
   'function': 'murmur3'},
  'vectorIndexConfig': {'skip': False,
   'cleanupIntervalSeconds': 300,
   'maxConnections': 64,
   'efConstruction': 128,
   'ef': -1,
   'dynamicEfMin': 100,
   'dynamicEfMax': 500,
   'dynamicEfFactor': 8,
   'vectorCacheMaxObjects': 1000000000000,
   'flatSearchCutoff': 40000,
   'distance': 'cosine',
   'pq': {'enabled': False,
    'bitCompression': False,
    'segments': 0,
    'centroids': 256,
    'encoder': {'type': 'kmeans', 'distribution': 'log-normal'

In [2]:
import os

In [98]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import os
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
      {"role": "user", "content":
           "I want you to become a Unix bash terminal scripting expert.\nOutput only the code needed to fulfill the task, add no explanations or descriptions of the involved procedure, including explantations before and after the code or script."
      },
      {"role": "user", "content": 'a script using wget that downloads a website and extract all links'}
  ],
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  # stop=["\n"]
)

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(outputs)

In [18]:
df.to_pickle("data/laws.pickle")

In [6]:
df = pd.read_pickle("data/laws.pickle")

""


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:0000:0100:01


In [4]:
import tiktoken
def get_number_tokens(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ").replace("\r", " ")
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ").replace("\r", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [7]:
df['tokens'] = df.content.apply(lambda x: get_number_tokens(str(x), model='text-embedding-ada-002'))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)

In [ ]:
df.tokens.sum()

In [9]:
((2782880 / 1000) * 0.004) 

11.13152

In [216]:
df_sub = df.sample(1)

In [218]:
df_sub['ada_embedding'] = df_sub.content.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)

1 § Den ränta som enligt 19 kap. 47 § socialförsäkringsbalken 
ska betalas på skuld som avser betalningsskyldighet för 
underhållsstöd enligt samma balk ska vara 0,14 procent för år 
2023.


In [219]:
len(df_sub["ada_embedding"].values[0])

1536

In [268]:
text = (df.loc[df.tokens > 40000].content.values[0])

In [357]:


chunks = split_text(text, chunk_size=2000, overlap=200)

In [358]:
df = pd.DataFrame(outputs)
df['content'] = df['content'].apply(lambda x: split_text(str(x), 15000, 200))
df = df.explode('content')

In [359]:
df['tokens'] = df.content.apply(lambda x: get_number_tokens(str(x), model='text-embedding-ada-002'))

In [360]:
df["tokens"].max()

5403

In [307]:
import re
split_pattern = r"\n\d+ kap"
items = re.split(split_pattern, text)[1:]
for item in items:
    print(len(item))

4847
7182
2437
18576
137
10838
11518
2902
7611
3625
9075
12948
1389
15775
3294
14817
2795
3229
1103
1429


In [290]:
print(text)

1 kap. Gemensamma bestämmelser

1 § Denna förordning har meddelats med stöd av 

- 2 § lagen (1994:1708) om EG:s förordningar om strukturstöd 
och om stöd till utveckling av landsbygden och 4 § lagen 
(1994:1710) om EG:s förordningar om jordbruksprodukter i fråga 
om 11 kap. 3-6 §§, 

- 3 a § lagen om EU:s förordningar om strukturstöd och om stöd 
till utveckling av landsbygden i fråga om 5 kap. 14 §,

- 3 § lagen om EG:s förordningar om jordbruksprodukter i fråga 
om 3 kap. 33 §§ och 6 kap. 17 §, 

- 6 § lagen om EU:s förordningar om strukturstöd och om stöd 
till utveckling av landsbygden och 10 § lagen om EG:s 
förordningar om jordbruksprodukter i fråga om 13 kap. 6 §, 

- 1 § lagen (2015:266) om lokala aktionsgrupper i fråga om 
3 kap. 11-16 §§, och 

- 8 kap. 7 § regeringsformen i fråga om övriga bestämmelser. 

2 § Denna förordning kompletterar 

1. Europaparlamentets och rådets förordning (EU) 2021/2115 av 
den 2 december 2021 om fastställande av regler om stöd för de 
strategis

In [305]:
len(items)

20

In [306]:
text

'1 kap. Gemensamma bestämmelser\r\n\r\n1 § Denna förordning har meddelats med stöd av \r\n\r\n- 2 § lagen (1994:1708) om EG:s förordningar om strukturstöd \r\noch om stöd till utveckling av landsbygden och 4 § lagen \r\n(1994:1710) om EG:s förordningar om jordbruksprodukter i fråga \r\nom 11 kap. 3-6 §§, \r\n\r\n- 3 a § lagen om EU:s förordningar om strukturstöd och om stöd \r\ntill utveckling av landsbygden i fråga om 5 kap. 14 §,\r\n\r\n- 3 § lagen om EG:s förordningar om jordbruksprodukter i fråga \r\nom 3 kap. 33 §§ och 6 kap. 17 §, \r\n\r\n- 6 § lagen om EU:s förordningar om strukturstöd och om stöd \r\ntill utveckling av landsbygden och 10 § lagen om EG:s \r\nförordningar om jordbruksprodukter i fråga om 13 kap. 6 §, \r\n\r\n- 1 § lagen (2015:266) om lokala aktionsgrupper i fråga om \r\n3 kap. 11-16 §§, och \r\n\r\n- 8 kap. 7 § regeringsformen i fråga om övriga bestämmelser. \r\n\r\n2 § Denna förordning kompletterar \r\n\r\n1. Europaparlamentets och rådets förordning (EU) 2021/21